# Batch segmentation with text prompts

This notebook shows how to generate object masks from text prompts with the Segment Anything Model (SAM). 

Working with Python 3.11.5 Environment. VS Code as Python IDE.

Reference: https://samgeo.gishub.org/examples/text_prompts/#segment-the-image

In [ ]:
# %pip install segment-geospatial groundingdino-py leafmap localtileserver gdal

In [ ]:
import leafmap
from samgeo import tms_to_geotiff
from samgeo.text_sam import LangSAM

In [ ]:
# Ackland, New Zealand (default)
latitude = -36.8900857
longitude = 174.7591024
zoom = 19

# render a map
m = leafmap.Map(center=[latitude, longitude], zoom=zoom, height="800px")
m.add_basemap("SATELLITE")
m

In [ ]:
bbox = m.user_roi_bounds()

coordinates = m.bounds

current_bbox = [coordinates[0][1], coordinates[0][0], coordinates[1][1], coordinates[1][0]]

print(current_bbox)
print(bbox, current_bbox)

if bbox is None:
    bbox = current_bbox
    print(bbox)

# Download tiles and generate image

image = "Image.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=zoom, source="Satellite", overwrite=True)

# Place the image on a map

m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

In [ ]:
sam = LangSAM()
text_prompt = "tree"

## Fine tunning

box_threshold: A higher value makes the model more selective = fewer overall detections. A lower value, conversely, makes the model more tolerant, leading to increased detections, including potentially less confident ones.

text_threshold: This value is used to associate the detected objects with the provided text prompt. A higher value requires a stronger association between the object and the text prompt, leading to more precise but potentially fewer associations. A lower value allows for looser associations, which could increase the number of associations but also introduce less precise matches.

In [ ]:
sam.predict(image, text_prompt, box_threshold=0.24, text_threshold=0.24)

In [ ]:
# Vegetation with boxes

sam.show_anns(
    cmap='Greens',
    box_color='red',
    title='Vegetation segmented with boxes',
    blend=True,
    output='trees_with_boxes'
)

# Vegetation

sam.show_anns(
    cmap='Greens',
    add_boxes=False,
    alpha=0.5,
    title='Vegetation segmented without boxes',
    output='trees_greens'
)

# Positive and Negative

sam.show_anns(
    cmap='Greys_r',
    add_boxes=False,
    alpha=1,
    title='Vegetation segmented grey scale',
    blend=False,
    output='trees_greyscale.tif',
)